In [1]:
# Core libraries
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from scipy.linalg import eigh



In [6]:
# ===========================
# Load and preprocess dataset
# ===========================

from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/Shared drives/GNN/RestingStateDataforAlex_3Networks.xlsx'
df = pd.read_excel(file_path)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#creating graphs


# Drop SID column and get all edge columns
edge_columns = df.columns.drop('SID')

# Extract all unique node names (stripped of 'ABS_' if present)
nodes_set = set()
for col in edge_columns:
    raw_node1, raw_node2 = col.split('-')
    node1 = raw_node1.replace('ABS_', '')
    node2 = raw_node2.replace('ABS_', '')
    nodes_set.add(node1)
    nodes_set.add(node2)

# Sort nodes for consistent ordering
nodes = sorted(list(nodes_set))
n_nodes = len(nodes)

# Map node name to index
node_to_idx = {node: i for i, node in enumerate(nodes)}

# Separate edges into ABS and non-ABS edges
abs_edge_cols = []
rel_edge_cols = []
abs_edge_to_idx = []
rel_edge_to_idx = []

for col in edge_columns:
    raw_node1, raw_node2 = col.split('-')
    node1 = raw_node1.replace('ABS_', '')
    node2 = raw_node2.replace('ABS_', '')
    idx1, idx2 = node_to_idx[node1], node_to_idx[node2]

    if 'ABS_' in col:
        abs_edge_cols.append(col)
        abs_edge_to_idx.append((idx1, idx2))
    else:
        rel_edge_cols.append(col)
        rel_edge_to_idx.append((idx1, idx2))

# Build adjacency matrices for each row
abs_adj_matrices = []
rel_adj_matrices = []

for _, row in df.iterrows():
    # Create zero matrices for ABS and relative graphs
    abs_adj = np.zeros((n_nodes, n_nodes))
    rel_adj = np.zeros((n_nodes, n_nodes))

    # Fill ABS adjacency matrix
    for col_idx, (i1, i2) in enumerate(abs_edge_to_idx):
        val = row[abs_edge_cols[col_idx]]
        abs_adj[i1, i2] = val
        abs_adj[i2, i1] = val  # symmetric

    # Fill relative adjacency matrix
    for col_idx, (i1, i2) in enumerate(rel_edge_to_idx):
        val = row[rel_edge_cols[col_idx]]
        rel_adj[i1, i2] = val
        rel_adj[i2, i1] = val  # symmetric

    abs_adj_matrices.append(abs_adj)
    rel_adj_matrices.append(rel_adj)


In [8]:
#making a dictionary of sid to absolute graph and relative graph

graphs_by_sid = {}

for sid, abs_mat, rel_mat in zip(df['SID'], abs_adj_matrices, rel_adj_matrices):
    graphs_by_sid[sid] = {
        'abs': abs_mat,
        'rel': rel_mat
    }

print(graphs_by_sid['epp348']['rel'])  # Relative graph
print(graphs_by_sid['epp348']['rel'].shape)

[[ 0.          0.6443512   0.60952988  0.48149378  0.21855582  0.02744795
   0.19705619 -0.14626492 -0.20716272 -0.01932991 -0.15652164 -0.28049982
  -0.17138803 -0.17873779 -0.05382542]
 [ 0.6443512   0.          0.53500292  0.64447851  0.71385829  0.0261051
   0.44132225 -0.55944315 -0.61724242 -0.10934934 -0.25394405 -0.15493399
  -0.18232735 -0.12858349 -0.11987415]
 [ 0.60952988  0.53500292  0.          0.50661799  0.37571928  0.42134627
   0.04587733 -0.12207047 -0.50220895 -0.05944264  0.08870163  0.1442684
  -0.41469532  0.02158323 -0.22569421]
 [ 0.48149378  0.64447851  0.50661799  0.          1.09910671  0.18484547
   0.21678365 -0.36616225 -0.53407179 -0.16535064 -0.03218564  0.05986651
  -0.11841703 -0.09234453 -0.26826495]
 [ 0.21855582  0.71385829  0.37571928  1.09910671  0.          0.10228209
   0.24679883 -0.45983477 -0.57815033 -0.17797306  0.07242114  0.08410655
  -0.13201032 -0.1261167  -0.20566083]
 [ 0.02744795  0.0261051   0.42134627  0.18484547  0.10228209  0.
 

In [9]:
#threshholding the connectivity graph

def threshold_graph(A, keep_fraction=0.4):
    triu_indices = np.triu_indices_from(A, k=1)                     #get upper triangle indices above the diagonal
    edge_values = A[triu_indices]                                   #get all edge values of upper triangle values
    threshold = np.percentile(edge_values, 100*(1-keep_fraction))   #60% weakest means keep top 40%
    mask = A >= threshold
    A_thresh = A * mask
    A_thresh = np.maximum(A_thresh, A_thresh.T)                     #ensure symmetry
    return A_thresh


for sid, graphs in graphs_by_sid.items():
    graphs_by_sid[sid]['abs'] = threshold_graph(graphs['abs'], keep_fraction=0.1)
    graphs_by_sid[sid]['rel'] = threshold_graph(graphs['rel'], keep_fraction=0.1)





In [10]:
#adding to dictionary demographic data and labels.


demo_cols = ['Age', 'handedness', 'sex', 'PrimaryEthnicity', 'PrimaryRace','Education','Parental Education']
outcome = ['Imp20PercentBPRS']
df_labels = pd.read_excel(file_path, sheet_name='outcomeanddemographics', skiprows=1)

demo_dict = df_labels.set_index('SID')[demo_cols].to_dict(orient='index')

graphs_by_sid = {}

for sid, abs_mat, rel_mat in zip(df['SID'], abs_adj_matrices, rel_adj_matrices):
    demos = demo_dict.get(sid, None)
    graphs_by_sid[sid] = {
        'abs': abs_mat,
        'rel': rel_mat,
        'demo': demos,
        'outcome': df_labels.loc[df_labels['SID'] == sid, outcome].values[0][0] if sid in df_labels['SID'].values else None
    }

#print(graphs_by_sid['epp270'])  # Relative graph

graphs_by_sid = {
    sid: graph
    for sid, graph in graphs_by_sid.items()
    if graph.get('outcome') is not None
}



outcome_count = sum(1 for graph in graphs_by_sid.values() if graph.get('outcome') == 1)
outcome_count_zero = sum(1 for graph in graphs_by_sid.values() if graph.get('outcome') == 0)
print(f"Number of instances with outcome == 1: {outcome_count}")
print(f"Number of instances with outcome == 0: {outcome_count_zero}")


#Number of instances with outcome == 1: 29
#Number of instances with outcome == 0: 27

Number of instances with outcome == 1: 29
Number of instances with outcome == 0: 27


In [11]:
import numpy as np
from scipy.linalg import eigh

def spectral_embedding(A, k=5):
    D = np.sum(A, axis=1)
    D_inv_sqrt = np.diag(1.0 / np.sqrt(D + 1e-10))
    L_sym = np.eye(len(A)) - D_inv_sqrt @ A @ D_inv_sqrt
    eigenvalues, eigenvectors = eigh(L_sym)
    return eigenvectors[:, 1:k+1]  # node embeddings shape (N_nodes, k)

for sid, graph in graphs_by_sid.items():
    A = graph['abs']  # or graph['rel']
    node_emb = spectral_embedding(A, k=5)  # shape (N_nodes, 5)
#    print(f"Graph {sid}: node_emb shape before pooling: {node_emb.shape}")

    graph_emb = np.mean(node_emb, axis=0)  # shape (5,)
#    print(f"Graph {sid}: graph_emb shape after pooling: {graph_emb.shape}")

    graphs_by_sid[sid]['graph_embedding'] = graph_emb


print(graphs_by_sid['epp270']['graph_embedding'].shape)  # Should print (5,)


# #
# Graph epp549: graph_emb shape after pooling: (5,)
# Graph epp551: node_emb shape before pooling: (15, 5)

(5,)


In [12]:
import torch
from torch.utils.data import Dataset

class EmbeddingDataset(Dataset):
    def __init__(self, sids, graphs_by_sid):
        self.sids = sids
        self.graphs_by_sid = graphs_by_sid

    def __len__(self):
        return len(self.sids)

    def __getitem__(self, idx):
        sid = self.sids[idx]
        embedding = torch.tensor(self.graphs_by_sid[sid]['graph_embedding'], dtype=torch.float)
        label = torch.tensor(self.graphs_by_sid[sid]['outcome'], dtype=torch.long)
        return embedding, label


In [13]:
import torch.nn.functional as F
import numpy as np

def mixup_embeddings(x1, y1, x2, y2, alpha=1.0, num_classes=2):
    lam = np.random.beta(alpha, alpha)
    x_mix = lam * x1 + (1 - lam) * x2
    y1_onehot = F.one_hot(y1, num_classes=num_classes).float()
    y2_onehot = F.one_hot(y2, num_classes=num_classes).float()
    y_mix = lam * y1_onehot + (1 - lam) * y2_onehot
    return x_mix, y_mix


In [14]:
import torch.nn as nn
import torch.nn.functional as F

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class BiggerMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [15]:
from torch.utils.data import DataLoader

def train_one_epoch(model, optimizer, dataloader, device, alpha=0.0, num_classes=2):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    for embeddings, labels in dataloader:
        embeddings, labels = embeddings.to(device), labels.to(device)

        indices = torch.randperm(embeddings.size(0))
        x2 = embeddings[indices]
        y2 = labels[indices]

        x_mix, y_mix = mixup_embeddings(embeddings, labels, x2, y2, alpha, num_classes)
        #x_mix, y_mix = x2,y2

        outputs = model(x_mix)
        loss = F.binary_cross_entropy_with_logits(outputs, y_mix)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds = outputs.argmax(dim=1)          # shape (batch_size,)
        true_labels = y_mix.argmax(dim=1)      # shape (batch_size,)


        true_labels = y_mix.argmax(dim=1)

        all_preds.append(preds.cpu())
        all_labels.append(true_labels.cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    train_acc = (all_preds == all_labels).float().mean().item()

    return total_loss / len(dataloader), train_acc



from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for embeddings, labels in dataloader:
            embeddings, labels = embeddings.to(device), labels.to(device)
            outputs = model(embeddings)  # logits shape: (batch_size, 2)
            preds = outputs.argmax(dim=1)  # predicted class 0 or 1

            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)  # default binary average

    return acc, f1



In [17]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sids = list(graphs_by_sid.keys())
labels = np.array([graphs_by_sid[sid]['outcome'] for sid in sids])
num_classes = len(set(labels))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

avg_acc = 0
avg_f1 = 0

for fold, (train_idx, test_idx) in enumerate(skf.split(sids, labels)):
    print(f"Fold {fold + 1}")

    train_sids = [sids[i] for i in train_idx]
    test_sids = [sids[i] for i in test_idx]

    train_dataset = EmbeddingDataset(train_sids, graphs_by_sid)
    test_dataset = EmbeddingDataset(test_sids, graphs_by_sid)

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=8)

    model = BiggerMLP(input_dim=graphs_by_sid[train_sids[0]]['graph_embedding'].shape[0],
                          hidden_dim=64, num_classes=num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(1, 10):
        loss, train_acc = train_one_epoch(model, optimizer, train_loader, device, alpha=1.0, num_classes=num_classes)
        print(f"Epoch {epoch} Loss: {loss:.4f} Train Accuracy: {train_acc:.4f}")

    test_acc, test_f1 = evaluate(model, test_loader, device)
    print(f"Fold {fold + 1} Test Accuracy: {test_acc:.4f}, F1 Score: {test_f1:.4f}")

    avg_acc += test_acc
    avg_f1 += test_f1

# Average across folds
avg_acc /= skf.get_n_splits()
avg_f1 /= skf.get_n_splits()

print(f"\nAverage Test Accuracy across folds: {avg_acc:.4f}")
print(f"Average Test F1 Score across folds: {avg_f1:.4f}")



Fold 1
Epoch 1 Loss: 0.7001 Train Accuracy: 0.4318
Epoch 2 Loss: 0.6938 Train Accuracy: 0.5227
Epoch 3 Loss: 0.6941 Train Accuracy: 0.5227
Epoch 4 Loss: 0.6926 Train Accuracy: 0.5227
Epoch 5 Loss: 0.6956 Train Accuracy: 0.5227
Epoch 6 Loss: 0.6949 Train Accuracy: 0.5227
Epoch 7 Loss: 0.6915 Train Accuracy: 0.5227
Epoch 8 Loss: 0.6940 Train Accuracy: 0.5227
Epoch 9 Loss: 0.6925 Train Accuracy: 0.5227
Fold 1 Test Accuracy: 0.5000, F1 Score: 0.6667
Fold 2
Epoch 1 Loss: 0.7007 Train Accuracy: 0.4444
Epoch 2 Loss: 0.6926 Train Accuracy: 0.5556
Epoch 3 Loss: 0.6969 Train Accuracy: 0.5111
Epoch 4 Loss: 0.6924 Train Accuracy: 0.5111
Epoch 5 Loss: 0.6926 Train Accuracy: 0.5111
Epoch 6 Loss: 0.6928 Train Accuracy: 0.5111
Epoch 7 Loss: 0.6930 Train Accuracy: 0.5111
Epoch 8 Loss: 0.6939 Train Accuracy: 0.5111
Epoch 9 Loss: 0.6933 Train Accuracy: 0.5111
Fold 2 Test Accuracy: 0.5455, F1 Score: 0.7059
Fold 3
Epoch 1 Loss: 0.7013 Train Accuracy: 0.4889
Epoch 2 Loss: 0.6944 Train Accuracy: 0.5111
Epoch